# Introduction to NoisyCircuits

## Library Imports

In [1]:
from NoisyCircuits import QuantumCircuit as QC
from NoisyCircuits.utils.GetNoiseModel import GetNoiseModel
import pickle
import os
import json

2025-11-24 14:21:19,383	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Setup Input Fields

In the input fields:

1. **token:** refers to the IBM API token for accessing IBM Quantum Systems. See IBM Documentation for account creation and token access [here](https://quantum.cloud.ibm.com/docs/en/guides/cloud-setup).
2. **backend_name:** refers to the IBM quantum hardware for which the noise model needs to be built. Currently, the software only supports the IBM Eagle R3 Chip set which have a basis gate set comprising of $X$, $\sqrt{X}$, $R_z(\cdot)$ and $ECR$. As of 15.08.2025, the backend codenamed "ibm_brisbane" is the only available quantum hardware with the Eagle R3 chip.
3. **num_qubits:** the number of qubits in the quantum circuits.
4. **num_cores:** the number of cores to run parallel Monte-Carlo Wavefunction (MCWF) Trajectories. (Ensure sufficient resources are available for runs.)
5. **num_trajectories:** the number of trajectories for the MCWF method.
6. **threshold:** the threshold for filtering out noise data.
7. **jsonize:** a boolean variable indicating whether the noise model needs to be jsonized or if the input noise model is already in a json format.

In [2]:
token = json.load(open(os.path.join(os.path.expanduser("~"), "ibm_api.json"), "r"))["apikey"] # Replace with your IBM Quantum token
backend_name = "ibm_fez"
num_qubits = 2
num_cores = 25
num_trajectories = 20
threshold = 1e-4
jsonize = True
verbose = True
# Options: "heron", "eagle". Note that "eagle" is now deprecated and only available in simulation mode (i.e., no noise model from hardware).
qpu_type = "heron" 

### Getting the Noise Model

Run the code below to obtain the noise model from IBM backend calibration data.

In [3]:
noise_model = GetNoiseModel(backend_name=backend_name, token=token).get_noise_model()

qiskit_runtime_service._discover_account:WARNING:2025-11-24 14:21:42,256: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-11-24 14:21:45,579: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Open_Sys. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-24 14:21:45,582: Using instance: Open_Sys, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-24 14:21:45,945: Using instance: Open_Sys, plan: open


Run the code below to use a sample noise model generated from IBM calibration data.

In [3]:
if qpu_type == "eagle":
    noise_model = pickle.load(open("../noise_models/Noise_Model_Eagle_QPU.pkl", "rb"))
elif qpu_type == "heron":
    noise_model = pickle.load(open("../noise_models/Noise_Model_Heron_QPU.pkl", "rb"))
else:
    raise ValueError("Invalid qpu_type. Choose either 'heron' or 'eagle'.")

### Initialize the Circuit Instance

In [4]:
nqc = QC(num_qubits=num_qubits, 
         noise_model=noise_model, 
         num_cores=num_cores,
         backend_qpu_type="heron", 
         num_trajectories=num_trajectories, 
         threshold=threshold, 
         jsonize=jsonize,
         verbose=verbose)

Completed Extraction of two-qubit gate Errors.
Starting post-processing on Single Qubit Errors.
Completed post-processing on Single Qubit Errors.
Processing two-qubit gate errors.
Qubit pair (0, 1): 18/48 errors above threshold (30 filtered out)
Qubit pair (1, 0): 18/48 errors above threshold (30 filtered out)
Two Qubit Gate errors processed.
Building Noise Operators for Two Qubit Gate Errors.


Completed building Noise Operators for Two Qubit Gate Errors.
Extracting Measurement Errors.
Available qubits in roerror_map: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
Requested qubits: [0, 1]
Completed Extraction of Measurement Errors.
Preparing Qubit Connectivity Map for Requested Qubits
Qubit Connectivity Map Prepared.
Returning Single Qubit Error Instructions, Two 

## Example Circuit

### Example 1: Creating an EPR Pair

First we create an EPR pair generating quantum circuit and execute it as a pure statevector simulation and as a noisy simulation (comparing density matrix and MCWF methods).

In [5]:
# Reset the circuit --> Ensure the circuit tape has no prior instructions
nqc.refresh()

In [6]:
# Create the EPR Pair Circuit
nqc.H(qubit=0)
nqc.CX(control=0, target=1)

In [7]:
# Run the circuit with pure state simulation, argument qubits is the measured qubits
nqc.run_pure_state(qubits=[0,1])

array([5.00000000e-01, 3.39184123e-33, 3.39184123e-33, 5.00000000e-01])

In [8]:
# Run the circuit with density matrix simulation
nqc.run_with_density_matrix(qubits=[0,1])

tensor([0.48032117, 0.02057075, 0.01992805, 0.47918003], requires_grad=True)

In [9]:
# Running with MCWF method, here the number of trajectories can be modified from the originally specified value
nqc.execute(qubits=[0,1], num_trajectories=100)

tensor([0.48256871, 0.01757185, 0.01756723, 0.48229221], requires_grad=True)

In [10]:
# MCWF converges to the output from the density matrix method as the number of trajectories increases, but at the cost of more computational time.
nqc.execute(qubits=[0,1], num_trajectories=1000)

tensor([0.48094121, 0.01919909, 0.01919402, 0.48066568], requires_grad=True)

For the MCWF method, using more trajectories ensures better convergence to the density matrix result.

### Example 2: Qubit Swap

In this example, we see the effect of noise on a qubit swap between two qubits whose states where initialized using angle encoding.

In [11]:
# nqc.refresh() resets the quantum circuit to zero gates.
# The argument to the rotation gates are the angle, followed by the qubit index.
nqc.refresh()
nqc.RY(theta=1.2, qubit=0)
nqc.RY(theta=0.5, qubit=1)
nqc.SWAP(qubit1=0, qubit2=1)

In [12]:
# Run with pure state simulation
nqc.run_pure_state(qubits=[0,1])

array([0.63948479, 0.29930649, 0.04169409, 0.01951463])

In [13]:
# Run with the density matrix simulation
nqc.run_with_density_matrix(qubits=[0,1])

tensor([0.61218666, 0.30444565, 0.05366529, 0.0297024 ], requires_grad=True)

In [14]:
# Run with MCWF Simulation
nqc.execute(qubits=[0,1], num_trajectories=500)

tensor([0.60631076, 0.30501247, 0.05658733, 0.03208945], requires_grad=True)

## Shutdown the Parallel instance

Due to parallel implementation with "ray", there is a requirement to explicitly shutdown the parallel pool

In [15]:
nqc.shutdown()